## 🚀 PySpark ETL Pipeline - Learning Exercise
### E-Commerce Data Analytics Use Case

---

#### 📋 Overview
This notebook is a hands-on exercise for learning PySpark ETL pipelines. You'll work with two datasets:
- **ecommerce_customers.csv**: Customer information
- **ecommerce_orders.csv**: Order transactions

#### 🎯 Your Mission
Build a complete ETL pipeline to analyze customer purchasing behavior and product performance.

#### ⚠️ Instructions
- Read each TODO carefully
- Write your code in the designated code cells
- Run cells sequentially
- Check your results after each step

#### ✅ TODO 1: Import Required Libraries and Create Spark session


In [30]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, sum, count, avg, round, when,
    datediff, lit, row_number, dense_rank
)
from pyspark.sql.window import Window

# create spark session 
spark = SparkSession.builder \
            .appName("E-Commerce ETL Pipeline") \
            .master("local[*]") \
            .getOrCreate()


#### ✅ TODO 2: Load Data - ecomm_customers and ecomm_orders

In [31]:
# load data 

customers_df = spark.read.csv("source_data/ecommerce_customers.csv", header=True, inferSchema=True)
orders_df = spark.read.csv("source_data/ecommerce_orders.csv", header=True, inferSchema=True)

customers_df.printSchema()
orders_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- signup_date: date (nullable = true)
 |-- status: string (nullable = true)

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- order_date: date (nullable = true)
 |-- status: string (nullable = true)



#### ✅ TODO 3: Check Data Quality and Perform Clean Up 
- [ ] Check for null values in all columns of both DataFrames
- [ ] Clean customers data:
  - [ ] Remove records with null email
  - [ ] Remove records with null country
  - [ ] Filter out "Invalid User" records
  - [ ] Remove duplicate `customer_id` values
- [ ] Store cleaned data in `customers_clean`
- [ ] Verify data quality improvements (print before/after counts)
- [ ] Add `total_amount` column to orders (quantity × price, rounded to 2 decimals)
- [ ] Store result in `orders_clean`

In [32]:
# Check Missing Records 

# Check for missing values in customers
print("Missing Values in Customers:")
customers_df.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in customers_df.columns
]).show()

# Check for missing values in orders
print("Missing Values in Orders:")
orders_df.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in orders_df.columns
]).show()


Missing Values in Customers:
+-----------+----+-----+-------+-----------+------+
|customer_id|name|email|country|signup_date|status|
+-----------+----+-----+-------+-----------+------+
|          0|   0|    1|      1|          0|     0|
+-----------+----+-----+-------+-----------+------+

Missing Values in Orders:
+--------+-----------+------------+--------+-----+----------+------+
|order_id|customer_id|product_name|quantity|price|order_date|status|
+--------+-----------+------------+--------+-----+----------+------+
|       0|          0|           0|       0|    0|         0|     0|
+--------+-----------+------------+--------+-----+----------+------+



In [33]:
# Clean customers data
customers_clean = customers_df \
    .filter(col("email").isNotNull()) \
    .filter(col("country").isNotNull()) \
    .filter(col("name") != "Invalid User") \
    .withColumn("email", col("email").cast("string")) \
    .dropDuplicates(["customer_id"])

print(f"Cleaned Customers: {customers_clean.count()} records (removed {customers_df.count() - customers_clean.count()})")


from pyspark.sql.types import DecimalType

orders_clean = orders_df \
    .filter(col("customer_id").isNotNull()) \
    .withColumn("total_amount", 
                round(col("quantity") * col("price"), 2).cast(DecimalType(10, 2)))

print(f"Cleaned Orders: {orders_clean.count()} records")

Cleaned Customers: 10 records (removed 2)
Cleaned Orders: 18 records


### ✅ TODO 4:TRANSFORM - Business Logic
#### Join data 
- [ ] Join `orders_clean` with `customers_clean` on `customer_id` (inner join)
- [ ] Store joined data in `customer_orders`
- [ ] Verify join results (check record count)
####  Add derived columns
Add the following columns to `customer_orders`:
- [ ] `order_month` - Extract YYYY-MM from `order_date` (first 7 characters)
- [ ] `is_high_value` - Categorize orders:
  - [ ] "High" if total_amount >= 500
  - [ ] "Medium" if total_amount >= 100
  - [ ] "Low" otherwise
- [ ] `days_since_signup` - Calculate days between `order_date` and `signup_date`
- [ ] Store enriched data in `customer_orders_enriched`
- [ ] Display sample enriched records


In [39]:
# join
# Rename status columns to be specific
orders_clean = orders_clean.withColumnRenamed("status", "order_status")
customers_clean = customers_clean.withColumnRenamed("status", "customer_status")

customer_orders = customers_clean.join(orders_clean, on="customer_id", how="inner")
print(f"Joined Dataset: {customer_orders.count()} records")

Joined Dataset: 16 records


In [40]:
# Derive columns 
customer_orders_enriched = customer_orders \
    .withColumn("order_month", col("order_date").substr(1, 7)) \
    .withColumn("is_high_value",  
                when(col("total_amount") >= 500, "High")
                .when(col("total_amount") >= 100, "Medium")
                .otherwise("Low")) \
    .withColumn("days_since_signup", datediff(col("order_date"), col("signup_date")))

customer_orders_enriched.show()

+-----------+----------------+--------------------+-------+-----------+---------------+--------+------------+--------+------+----------+------------+------------+-----------+-------------+-----------------+
|customer_id|            name|               email|country|signup_date|customer_status|order_id|product_name|quantity| price|order_date|order_status|total_amount|order_month|is_high_value|days_since_signup|
+-----------+----------------+--------------------+-------+-----------+---------------+--------+------------+--------+------+----------+------------+------------+-----------+-------------+-----------------+
|          1|      John Smith|john.smith@email.com|    USA| 2024-01-15|         active|    1014|   USB Cable|      10| 12.99|2024-03-28|   completed|      129.90|    2024-03|       Medium|               73|
|          1|      John Smith|john.smith@email.com|    USA| 2024-01-15|         active|    1003|    Keyboard|       1| 79.99|2024-03-17|   completed|       79.99|    2024-0

### TODO 5:TRANSFORM - Business Logic Customer Analytics (Aggregation)
Create `customer_summary` with:
- [ ] Group by: `customer_id`, `name`, `email`, `country`, `status`
- [ ] Calculate aggregations:
  - [ ] `total_orders` - Count of all orders
  - [ ] `total_spent` - Sum of total_amount (rounded to 2 decimals)
  - [ ] `avg_order_value` - Average total_amount (rounded to 2 decimals)
  - [ ] `completed_orders` - Count of orders with status='completed'
- [ ] Sort by `total_spent` descending
- [ ] Display top 10 customers

In [43]:

# Customer-level aggregations
customer_summary = customer_orders_enriched \
    .groupBy("customer_id", "name", "email", "country", "customer_status") \
    .agg(
        count("order_id").alias("total_orders"),
        sum("total_amount").alias("total_spent"),
        avg("total_amount").alias("avg_order_value"),
        count(when(col("order_status") == "completed", 1)).alias("completed_orders")
    ) \
    .withColumn("total_spent", round(col("total_spent"), 2)) \
    .withColumn("avg_order_value", round(col("avg_order_value"), 2)) \
    .orderBy(col("total_spent").desc())

print("Customer Summary Statistics:")
customer_summary.show(10)

Customer Summary Statistics:
+-----------+----------------+--------------------+-------+---------------+------------+-----------+---------------+----------------+
|customer_id|            name|               email|country|customer_status|total_orders|total_spent|avg_order_value|completed_orders|
+-----------+----------------+--------------------+-------+---------------+------------+-----------+---------------+----------------+
|          8|     Yuki Tanaka|    yuki.t@email.com|  Japan|         active|           2|    2299.97|        1149.99|               2|
|          4|   Sarah Johnson|   sarah.j@email.com|    USA|         active|           2|    1599.97|         799.99|               2|
|          1|      John Smith|john.smith@email.com|    USA|         active|           3|    1209.88|         403.29|               3|
|          6|     Emma Wilson|    emma.w@email.com|     UK|         active|           2|    1064.94|         532.47|               1|
|          3|          Li Wei|   